In [42]:
import pandas as pd
import requests
import time
import random
import re
from faker import Faker

random.seed(42)
Faker.seed(42)
fake = Faker('es_MX')

In [43]:
df_destinos = pd.read_csv('../destinos/destinos.csv')
df_destinos

,nombre,categoria,tipo,ciudad,municipio,estado,codigopostal,lat,lon,min_lat,max_lat,min_lon,max_lon,id_estado,id_municipio
0,Cabo San Lucas,place,city,Cabo San Lucas,Los Cabos,Baja California Sur,23470,22.893888,-109.920060,22.733888,23.053888,-110.080060,-109.760060,3,3008
1,Cancún,boundary,administrative,Cancún,Benito Juárez,Quintana Roo,unkwown,21.152747,-86.842576,20.970585,21.212375,-86.995617,-86.740526,23,23005
2,Loreto,place,town,unkwown,Loreto,Baja California Sur,23880,26.009791,-111.345252,25.969791,26.049791,-111.385252,-111.305252,3,3009
3,San José del Cabo,place,city,San José del Cabo,Los Cabos,Baja California Sur,23400,23.059836,-109.702515,22.899836,23.219836,-109.862515,-109.542515,3,3008
4,Ixtapa,boundary,administrative,unkwown,Ixtapa,Chiapas,unkwown,16.807623,-92.898650,16.684633,16.930706,-93.020620,-92.811072,7,7044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Tecozautla,boundary,administrative,unkwown,Tecozautla,Hidalgo,unkwown,20.539699,-99.652609,20.420234,20.659132,-99.835080,-99.449233,13,13059
109,Zempoala,boundary,administrative,unkwown,Zempoala,Hidalgo,unkwown,19.925944,-98.643350,19.813184,20.038677,-98.830983,-98.528064,13,13083
110,Coatepec,boundary,administrative,unkwown,unkwown,Veracruz,unkwown,19.454313,-96.959637,19.357274,19.532391,-97.106230,-96.797034,30,0
111,Orizaba,place,city,Orizaba,Orizaba,Veracruz,94300,18.849737,-97.106006,18.689737,19.009737,-97.266006,-96.946006,30,30118


In [44]:
def buscar_hospedajes_en_direccion(ciudad, estado, pais="Mexico", espera=1):
    """
    Consulta la API de Nominatim con datos de dirección y devuelve la respuesta JSON.
    
    Parámetros:
        calle (str): Nombre de la calle (sin número interior).
        ciudad (str): Nombre del municipio o ciudad.
        estado (str): Estado de la República.
        pais (str): País (por defecto 'Mexico').
        espera (int): Tiempo de espera entre solicitudes (segundos).
        
    Retorna:
        dict | None: Primer resultado de la búsqueda o None si no hay resultados.
    """
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "amenity": 'Hoteles en', # Obtiene hoteles cercanos a la dirección provista.
        "city": ciudad,
        "state": estado,
        "country": pais,
        "format": "jsonv2",
        "addressdetails": 1,
        "extratags": 1,
    }

    headers = {
        'User-Agent': 'metabuscador-dtl',  # Si no se pone un user-agent se enoja la API
        'Accept': 'application/json',  # Para aceptar respuesta en JSON
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        resultados = response.json()

        if resultados:
            print(resultados)
            return resultados
        else:
            print('Sin resultados')
            return None
    except requests.RequestException as e:
        print(f"Error en la solicitud: {e}")
        return None
    finally:
        time.sleep(espera)  # Para evitar ser bloqueado por exceso de solicitudes

def generar_hoteles_con_direccion(fila):
    print(fila['nombre'], fila['estado'])
    resultados = buscar_hospedajes_en_direccion(fila['nombre'], fila['estado'])
    if resultados is not None:
        for resultado in resultados:
            hotel = dict()
            propietario = dict()
            direccion = dict()

            hotel['nombre'] = 'Hotel ' + fake.company() if resultado.get('name') == '' else re.sub(r"[^\w\s,]", '', resultado.get('name'), flags=re.UNICODE)
            hotel['precio'] = random.randint(1000, 20000)
            hotel['destino_id'] = fila.name+1

            propietario['nombre'] = fake.name()
            propietario['telefono'] = fake.phone_number()
            propietario['email'] = fake.email()

            direccion['nombre'] = resultado.get('address').get('road')
            direccion['estado'] = resultado.get('address').get('state')
            direccion['municipio'] = resultado.get('address').get('county')
            direccion['ciudad'] = resultado.get('address').get('city')
            direccion['cp'] = resultado.get('address').get('postcode')
            direccion['lat'] = resultado.get('lat')
            direccion['lon'] = resultado.get('lon')

            hoteles.append(hotel)
            propietarios.append(propietario)
            direcciones.append(direccion)

In [45]:
hoteles = list()
propietarios = list()
direcciones = list()

df_destinos.apply(generar_hoteles_con_direccion, axis=1)

Cabo San Lucas Baja California Sur
[{'place_id': 291883654, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 5321213721, 'lat': '22.8982431', 'lon': '-109.9191241', 'category': 'tourism', 'type': 'hotel', 'place_rank': 30, 'importance': -0.005253108882050238, 'addresstype': 'tourism', 'name': 'Hotel Pericue', 'display_name': 'Hotel Pericue, Calle José María Morelos y Pavón, Mariano Matamoros, Cabo San Lucas, Municipio de Los Cabos, Baja California Sur, 23470, México', 'address': {'tourism': 'Hotel Pericue', 'road': 'Calle José María Morelos y Pavón', 'suburb': 'Mariano Matamoros', 'city': 'Cabo San Lucas', 'county': 'Municipio de Los Cabos', 'state': 'Baja California Sur', 'ISO3166-2-lvl4': 'MX-BCS', 'postcode': '23470', 'country': 'México', 'country_code': 'mx'}, 'extratags': None, 'boundingbox': ['22.8981931', '22.8982931', '-109.9191741', '-109.9190741']}, {'place_id': 290458205, 'licence': 'Data © OpenStreetMap contri

0      None
1      None
2      None
3      None
4      None
       ... 
108    None
109    None
110    None
111    None
112    None
Length: 113, dtype: object

In [46]:
df_hoteles = pd.DataFrame(hoteles)
df_direcciones = pd.DataFrame(direcciones)
df_propietarios = pd.DataFrame(propietarios)

In [47]:
df_hoteles.to_csv('hospedajes_hoteles_de_destinos.csv', index=False)
df_propietarios.to_csv('propietarios_hoteles_de_destinos.csv', index=False)
df_direcciones.to_csv('direcciones_hoteles_de_destinos.csv', index=False)